## Predicting the final grade of a student

This data approach student achievement in secondary education of two Portuguese schools. The data attributes include student grades, demographic, social and school-related features) and it was collected by using school reports and questionnaires. Two datasets are provided regarding the performance in two distinct subjects: Mathematics (mat) and Portuguese language (por). In [Cortez and Silva, 2008], the two datasets were modeled under binary/five-level classification and regression tasks. Important note: the target attribute G3 has a strong correlation with attributes G2 and G1. This occurs because G3 is the final year grade (issued at the 3rd period), while G1 and G2 correspond to the 1st and 2nd period grades. It is more difficult to predict G3 without G2 and G1, but such prediction is much more useful (see paper source for more details).

**Attribute Information**:
- school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
- sex - student's sex (binary: 'F' - female or 'M' - male)
- age - student's age (numeric: from 15 to 22)
- address - student's home address type (binary: 'U' - urban or 'R' - rural)
- famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
- Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
- Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
- Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
- Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
- Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
- reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
- guardian - student's guardian (nominal: 'mother', 'father' or 'other')
- traveltime - home to school travel time (numeric: 1 - 1 hour)
- studytime - weekly study time (numeric: 1 - 10 hours)
- failures - number of past class failures (numeric: n if 1<=n<3, else 4)
- schoolsup - extra educational support (binary: yes or no)
- famsup - family educational support (binary: yes or no)
- paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
- activities - extra-curricular activities (binary: yes or no)
- nursery - attended nursery school (binary: yes or no)
- higher - wants to take higher education (binary: yes or no)
- internet - Internet access at home (binary: yes or no)
- romantic - with a romantic relationship (binary: yes or no)
- famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
- freetime - free time after school (numeric: from 1 - very low to 5 - very high)
- goout - going out with friends (numeric: from 1 - very low to 5 - very high)
- Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
- Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
- health - current health status (numeric: from 1 - very bad to 5 - very good)
- absences - number of school absences (numeric: from 0 to 93)
**these grades are related with the course subject, Math or Portuguese:**
- G1 - first period grade (numeric: from 0 to 20)
- G2 - second period grade (numeric: from 0 to 20)
- G3 - final grade (numeric: from 0 to 20, output target)
**Relevant Papers:**
P. Cortez and A. Silva. Using Data Mining to Predict Secondary School Student Performance. In A. Brito and J. Teixeira Eds., Proceedings of 5th FUture BUsiness TEChnology Conference (FUBUTEC 2008) pp. 5-12, Porto, Portugal, April, 2008, EUROSIS, ISBN 978-9077381-39-7.
Available at: <a href='http://www3.dsi.uminho.pt/pcortez/student.pdf'>Web Link</a>



**This notebook based on this kaggle notebook:** **<a href='https://www.kaggle.com/dipam7/introduction-to-eda-and-machine-learning#Modeling'>WEB LINK</a>**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_error, accuracy_score


from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from eli5.sklearn import PermutationImportance
import eli5
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


from sklearn.pipeline import make_pipeline
%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_csv('../input/student-grade-prediction/student-mat.csv')

In [ ]:
df

In [ ]:
df.head()

In [ ]:
uniques = pd.DataFrame(columns=["Feature", "Uniques", "num_unique"])

In [ ]:
for i in range(len(df.columns)):
    uniques.loc[i] = [df.columns[i]] + [df[df.columns[i]].unique()] + [df[df.columns[i]].nunique()]

In [ ]:
uniques

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

In [ ]:
df['mean_periods'] = (df.G1 + df.G2 + df.G3) / 3

In [ ]:
plt.hist(df.mean_periods)

In [ ]:
cols_obj = list(df.dtypes[df.dtypes == object].index[0:])
cols_obj.pop(1)
i=1
plt.figure(figsize=[15,20])
y, hue = 'proportion', 'sex'

for f in cols_obj:
    plt.subplot(8,2,i)
    df[[f, 'sex']]\
       .value_counts(normalize=True)\
       .rename(y)\
       .reset_index()\
       .pipe((sns.barplot, "data"), x=f, y='proportion', hue='sex', alpha=0.8)
    plt.title(f'Proportion of {f}')
    i+=1
plt.tight_layout()



In [ ]:
cols_cont = [column for column in df.columns if column not in cols_obj and 
             column not in ['sex', 'Medu','Fedu','failures',]]

In [ ]:
plt.figure(figsize=[25, 50])
i = 1

for x in cols_cont:
    plt.subplot(14,2,i)
    sns.boxplot(x='G3', y=x, data=df)
    i+=1
    plt.title(f'Distribution of {x}')

Boys spend less time on studying than girls

In [ ]:
sns.histplot(data=df, x='studytime', y='mean_periods', hue='sex')

In general boys study better than girls

In [ ]:
df.groupby('sex')['mean_periods'].mean()

In [ ]:
sns.countplot(data=df, x='G3')

In [ ]:
sns.displot(x='G3',
           hue='sex', 
           data=df,
           stat='probability')

Internet improves your marks!

In [ ]:
sns.displot(x='G3',
           hue='internet', 
           data=df,
           stat='probability')

Students who don't take paid classes have better marks

In [ ]:
plt.figure(figsize=[12,8])
sns.boxplot(x='paid', y='G3', data=df)

Usually student's parents consume alcohol on weekends. But boys parents like to drink during workdays and parents of boys consume more alcohol in general

In [ ]:
alco = ['Dalc', 'Walc']
plt.figure(figsize=[16,12])
i = 1

for x in alco:
    plt.subplot(1,2,i)
    sns.boxplot(x='sex', y=x, data=df)
    i+=1
    plt.title(f'Distribution of {x}')

In [ ]:
sns.countplot(df.absences)

Boys and Girls absence school identically. But among girls there are some outliers

In [ ]:
sns.boxplot(x='sex', y='absences', data=df)

In [ ]:
cols_num = list(df.dtypes[df.dtypes == 'int64'].index[0:])

In [ ]:
def correlation_heatmap(train):
    correlations = train.corr()

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
    plt.show();
    
correlation_heatmap(df[cols_num])

From corr matrix we can see tha Fedu and Medu, goout and Walc, Walc and Dalc correlate a lot, we need to delete one of the pair to use in Learning models that can be affected by correlation.

In [ ]:
sns.jointplot(x='Fedu', y='Medu', 
              data=df, kind='scatter', hue='sex');

In [ ]:
sns.jointplot(x='goout', y='Walc', 
              data=df, kind='scatter', hue='G3');

### Encoding variables

In [ ]:
category_df = df.select_dtypes(include=['object'])

In [ ]:
df_dummy = pd.get_dummies(category_df)

In [ ]:
df_dummy['G3'] = df['G3']

Correlation of encoded variables

In [ ]:
df_dummy.corr()['G3'].sort_values(ascending=False)

We drop G1 and G2 and school because they highly correlate with G3

In [ ]:
y = df['G3']

X = df.drop(['G1', 'G2', 'mean_periods', 'school'], axis=1)

X = pd.get_dummies(X)

In [ ]:
most_correlated = X.corr().abs()['G3'].sort_values(ascending=False)


In [ ]:
most_correlated = most_correlated[:9]
most_correlated

In [ ]:
X = X.loc[:, most_correlated.index]
X

**Students whose parents have better education score higher**

In [ ]:
sns.scatterplot(x='Medu', y='G3', data=X)

In [ ]:
sns.scatterplot(x='Fedu', y='G3', data=X)

**Age does not affect score**

In [ ]:
sns.scatterplot(data=X, x='age', y='G3')

**Students with less failures score more**

In [ ]:
sns.boxplot(data=X, x='failures', y='G3')

**Students who go out less score better**

In [ ]:
sns.countplot(data=X, x='goout')

In [ ]:
sns.boxplot(data=X, x='goout', y='G3')

**Students who want to take higher education score better**

In [ ]:
sns.countplot(data=X, x='higher_yes')

In [ ]:
sns.boxplot(data=X, x='higher_yes', y='G3')

**Students who do not have romantic relat. score better**

In [ ]:
sns.countplot(data=X, x='romantic_no')

In [ ]:
sns.boxplot(data=X, x='romantic_no', y='G3')

hypothesis:
Students who score high marks :
- parents have good education
- have <2 failures
- go out <= 3 times a week
- want to take higher education
- dont have romantic relationship 

### Modeling

Predicting G3 through regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=17)

In [ ]:
X_train = X_train.drop('G3', axis=1)
X_test = X_test.drop('G3', axis=1)

In [ ]:
models_df =  pd.DataFrame(columns=['mae', 'rmse', 'accuracy'])

In [ ]:
def make_report(models_df, model, X_test, y_test, name):
    report = pd.DataFrame(columns={'mae'}, data=[0])
    
    report['mae'] = mean_absolute_error(y_test, model.predict(X_test).round())
    report['rmse'] = np.sqrt(mean_squared_error(y_test, model.predict(X_test).round()))
    report['accuracy'] = accuracy_score(y_test, model.predict(X_test).round())
    
    report.index = [name]
    models_df = models_df.append(report)
    return models_df

**Raw Lofistic Regression**

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
models_df = make_report(models_df, lr, X_test, y_test, 'Logistic_regression')

In [ ]:
models_df

**LR GridSearchCV**

In [ ]:
%%time
kf = KFold(n_splits=5,shuffle=True, random_state=17)
lr = LogisticRegression()
params = {
    'C': np.logspace(-6, 6, 100),
    'random_state': [17],
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': range(100, 1001, 100),
}

lr_gs = GridSearchCV(lr, param_grid=params, cv=kf, verbose=True, n_jobs=-1)
lr_gs.fit(X_train, y_train)

In [ ]:
lr_gs = lr_gs.best_estimator_


In [ ]:
lr_gs.get_params()

In [ ]:
models_df = make_report(models_df, lr_gs, X_test, y_test, 'Logistic_regression_GridSearchCV')

In [ ]:
models_df

**Linear Regression Raw**

In [ ]:
linear = LinearRegression()
linear.fit(X_train, y_train)

In [ ]:
models_df = make_report(models_df, linear, X_test, y_test, 'Linear_regression')

In [ ]:
models_df

**Random Forest Raw**

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

In [ ]:
models_df = make_report(models_df, rf, X_test, y_test, 'RandomForest')

In [ ]:
models_df

**Random Forest GridSearchCV**

In [ ]:
%%time
kf = KFold(n_splits=3,shuffle=True, random_state=17)
rf = RandomForestRegressor()

params = {
    'n_estimators': range(100, 1001, 100),
    'max_depth': range(3,15),
    'min_samples_split': range(2, 6),
    'min_samples_leaf': range(1, 6),
    'random_state': [17]
}

rf_gs = GridSearchCV(rf, param_grid=params, verbose=True, cv=kf, n_jobs=-1)
rf_gs.fit(X_train, y_train)

In [ ]:
rf_gs = rf_gs.best_estimator_

In [ ]:
models_df = make_report(models_df, rf_gs, X_test, y_test, 'RandomForest_GridSearchCCV')

In [ ]:
models_df

**Decision Tree Raw**

In [ ]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

In [ ]:
models_df = make_report(models_df, tree, X_test, y_test, 'Decision Tree')

In [ ]:
models_df

**Decision Tree GridSearchCV**

In [ ]:
%%time
kf = KFold(n_splits=3,shuffle=True, random_state=17)
tree = DecisionTreeRegressor()
params = {
    'criterion': ['mse', 'mae'],
    'max_depth': range(3,15),
    'min_samples_split': range(2, 6),
    'min_samples_leaf': range(1, 6),
}

tree_gs = GridSearchCV(tree, param_grid=params, cv=kf, verbose=2, n_jobs=-1)
tree_gs.fit(X_train, y_train)

In [ ]:
models_df = make_report(models_df, tree_gs, X_test, y_test, 'Decision_Tree_GridSearchCV')

In [ ]:
models_df

In [ ]:
plt.figure(figsize=(20, 12))

# Root mean squared error
ax =  plt.subplot(1, 3, 1)
models_df.sort_values('mae', ascending = True).plot.bar(y = 'mae', color = 'b', ax = ax, fontsize=20)
plt.title('Model Mean Absolute Error', fontsize=20) 
plt.ylabel('MAE', fontsize=20)

# Median absolute percentage error
ax = plt.subplot(1, 3, 2)
models_df.sort_values('rmse', ascending = True).plot.bar(y = 'rmse', color = 'r', ax = ax, fontsize=20)
plt.title('Model Root Mean Squared Error', fontsize=20) 
plt.ylabel('RMSE',fontsize=20)

ax = plt.subplot(1, 3, 3)
models_df.sort_values('accuracy', ascending = True).plot.bar(y = 'accuracy', color = 'g', ax = ax, fontsize=20)
plt.title('Accuracy', fontsize=20) 
plt.ylabel('ACCURACY',fontsize=20)

plt.show()

As we can see, on these features, Linear regression get the best MAE and RMSE, but the best accuracy is Logistic regression with tunned params.

**Let's see what results we will get if we use all features, except G1 and G2**

### Modeling with all features

In [ ]:
X_all = df.drop(['G1', 'G2', 'G3', 'mean_periods'], axis = 1)

**Scaled features**

In [ ]:
X_scaled = X_all

In [ ]:
X_scaled[['age', 'absences']] = MinMaxScaler(feature_range=(1,4)).fit_transform(X_all[['age', 'absences']])

In [ ]:
X_scaled = pd.get_dummies(X_scaled)

In [ ]:
X_scaled

In [ ]:
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y, test_size=.3, random_state=17)

**Logistic Regression**

In [ ]:
params = {'C': 0.02310129700083158,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 17,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False
}

lr = LogisticRegression(**params)
lr.fit(X_train_scaled, y_train_scaled)

In [ ]:
models_df = make_report(models_df, lr, X_test_scaled, y_test_scaled, 'Logistic_regression_all_scaled')

In [ ]:
models_df

**Linear Regression All scaled**

In [ ]:
linear = LinearRegression()
linear.fit(X_train_scaled, y_train_scaled)

In [ ]:
models_df = make_report(models_df, linear, X_test_scaled, y_test_scaled, 'Linear_regression_all_scaled')

In [ ]:
models_df

**Unscaled features**

In [ ]:
X_train_unscaled, X_test_unscaled, y_train_unscaled, y_test_unscaled = train_test_split(pd.get_dummies(X_all), y, test_size=.3, random_state=42)

**Logistic Regression**

In [ ]:
params = {'C': 0.02310129700083158,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 17,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False
}

lr = LogisticRegression(**params)
lr.fit(X_train_unscaled, y_train_unscaled)

In [ ]:
models_df = make_report(models_df, lr, X_test_unscaled, y_test_unscaled, 'Logistic_regression_all_unscaled')

In [ ]:
models_df

**Linear regression**

In [ ]:
linear = LinearRegression()
linear.fit(X_train_unscaled, y_train_unscaled)

In [ ]:
models_df = make_report(models_df, linear, X_test_unscaled, y_test_unscaled, 'Linear_regression_all_unscaled')

In [ ]:
models_df

**We've got different answers. Maybe it is because of seed, let's set seed to 17**

**Unscaled features(with same seed as scaled)**

In [ ]:
X_train_unscaled, X_test_unscaled, y_train_unscaled, y_test_unscaled = train_test_split(pd.get_dummies(X_all), y, test_size=.3, random_state=17)

**Logistic Regression**

In [ ]:
params = {'C': 0.02310129700083158,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 17,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False
}

lr = LogisticRegression(**params)
lr.fit(X_train_unscaled, y_train_unscaled)

In [ ]:
models_df = make_report(models_df, lr, X_test_unscaled, y_test_unscaled, 'Logistic_regression_all_unscaled_same_seed')

In [ ]:
models_df

**Linear regression**

In [ ]:
linear = LinearRegression()
linear.fit(X_train_unscaled, y_train_unscaled)

In [ ]:
models_df = make_report(models_df, linear, X_test_unscaled, y_test_unscaled, 'Linear_regression_all_unscaled_same_seed')

In [ ]:
models_df

In [ ]:
plt.figure(figsize=(25, 14))

# Root mean squared error
ax =  plt.subplot(1, 3, 1)
models_df.sort_values('mae', ascending = True).plot.bar(y = 'mae', color = 'b', ax = ax, fontsize=20)
plt.title('Model Mean Absolute Error', fontsize=20) 
plt.ylabel('MAE', fontsize=20)

# Median absolute percentage error
ax = plt.subplot(1, 3, 2)
models_df.sort_values('rmse', ascending = True).plot.bar(y = 'rmse', color = 'r', ax = ax, fontsize=20)
plt.title('Model Root Mean Squared Error', fontsize=20) 
plt.ylabel('RMSE',fontsize=20)

ax = plt.subplot(1, 3, 3)
models_df.sort_values('accuracy', ascending = True).plot.bar(y = 'accuracy', color = 'g', ax = ax, fontsize=20)
plt.title('Accuracy', fontsize=20) 
plt.ylabel('ACCURACY',fontsize=20)

plt.show()

**Best model : LogisticRegression with all features, no matter scaled or unscaled**

### Feature importance

In [ ]:
perm = PermutationImportance(lr, scoring='neg_root_mean_squared_error').fit(X_test_unscaled, y_test_unscaled)

In [ ]:
eli5.show_weights(perm, feature_names=X_test_unscaled.columns.tolist())

In [ ]:
perm_importance = eli5.explain_weights_df(perm).sort_values(by='weight',
                                                            ascending=False)

In [ ]:
perm_importance = perm_importance[perm_importance['weight'] > 0]
perm_importance['f'] = perm_importance['feature'].apply(lambda x: int(x[1:]))
cols_perm = list(X_test_unscaled.columns[perm_importance['f']])
perm_importance['feature'] = cols_perm

In [ ]:
perm_importance[:10]

The hypothesis was confirmed, failures, Fedu and Walc influence the final score.
Also traveltime and quality of family relationship